# Title
We are attempting to solve this kaggle challenge: https://www.kaggle.com/competitions/titanic/data

### Imports and converting file to dataframe

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [87]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

In [69]:

train = pd.read_csv("train.csv")
test = pd.read_csv("train.csv")

count = 0
for i in train["Survived"]:
    if i==1:
        count +=1 

print("Survival rate:", count/len(train["Survived"]))
print("The regular success rate should work fine")

le = len(train)
train1 = train.dropna()
print("data length", le, "-->", len(train1), "if we dropna. Not a good idea.")
print()
# print(train.head(5))

Survival rate: 0.3838383838383838
The regular success rate should work fine
data length 891 --> 183 if we dropna. Not a good idea.



### Cleaning the data 
annoying!! (sadface)

I referred to https://www.kaggle.com/code/murtadhanajim/80-in-titanic-dataset-using-random-forests/notebook but just for 3 lines

In [100]:
def process_data(df):
    df = df.drop(columns=["PassengerId", "Name", "Ticket"]) 
    # ^^ dropped because i don't know how to deal with these...
    # there's probably a way to get info from the ticket number though?

    '''replace missing data with median of the column'''
    ''' Reference: https://www.kaggle.com/code/murtadhanajim/80-in-titanic-dataset-using-random-forests/notebook'''

    # Fill missing embarked with the most frequent value
    df['Embarked'].fillna(df['Embarked'].mode()[0])

    # Fill missing fare with the median
    df['Fare'].fillna(df['Fare'].median())

    df['Pclass'].fillna(df['Pclass'].median())
    df['Age'].fillna(df['Age'].median())
    df['SibSp'].fillna(df['SibSp'].median())
    df['Parch'].fillna(df['Parch'].median())

    # Convert cabin to first letter and fill missing values with 'N'
    df['Cabin'] = df['Cabin'].fillna('N').map(lambda x: x[0])

    df['Sex'] = df['Sex'].fillna('N').map(lambda x: x[0])

    df = df.drop(columns = ["Survived"]) #This is y
    df.dropna()
    return df

df = process_data(train)
test = process_data(train)

# '''Scale the age'''
# scaler = StandardScaler()
# df["Age"] = scaler.fit_transform(df[["Age"]]) #double brackets so that we pass in a dataframe? Not sure why
# test["Age"] = scaler.transform(test[["Age"]])
print(df.head())
print(test.head())
print(test.shape)

   Pclass Sex   Age  SibSp  Parch     Fare Cabin Embarked
0       3   m  22.0      1      0   7.2500     N        S
1       1   f  38.0      1      0  71.2833     C        C
2       3   f  26.0      0      0   7.9250     N        S
3       1   f  35.0      1      0  53.1000     C        S
4       3   m  35.0      0      0   8.0500     N        S
   Pclass Sex   Age  SibSp  Parch     Fare Cabin Embarked
0       3   m  22.0      1      0   7.2500     N        S
1       1   f  38.0      1      0  71.2833     C        C
2       3   f  26.0      0      0   7.9250     N        S
3       1   f  35.0      1      0  53.1000     C        S
4       3   m  35.0      0      0   8.0500     N        S
(891, 8)


In [101]:
'''Says CHATGPT'''
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define columns to be transformed
numeric_cols = ["Age", "Fare"]
categorical_cols = ["Sex", "Embarked", "Cabin", "Pclass"]

# Create the ColumnTransformer
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

# Fit and transform the training data
X_train = preprocessor.fit_transform(df)

# Transform the test data
X_test = preprocessor.transform(test)

y_train = train["Survived"]

print(X_train.shape)


(891, 20)


### Fitting to model

In [ ]:
#KNN
knn = KNeighborsRegressor()
mod = GridSearchCV(estimator = knn,
             param_grid = {'n_neighbors': list(range(20))})
mod.fit(X_train,y_train)
print(pd.DataFrame(mod.cv_results_))

ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/for_phone/opt/anaconda3/envs/sklearn-env/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/for_phone/opt/anaconda3/envs/sklearn-env/lib/python3.13/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/for_phone/opt/anaconda3/envs/sklearn-env/lib/python3.13/site-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~